In [ ]:
%matplotlib qt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from helpers.ssa import SSA
from helpers.blackbox_gyro import init_figure_settings,show_axes, show_axes_v2,calc_max_noise_regression

init_figure_settings()


In [ ]:
#blackbox_decode btfl_001.bbl 
data = pd.read_csv('data\\btfl_001.bbl.csv', encoding='cp1251', decimal='.').rename(columns=lambda x: x.strip())
print(data.columns)

In [ ]:
data['time (us)'] = (data['time (us)'] - data['time (us)'][0])/1000000
#print(data.iloc[0,:].values.dtype)
print(data.shape)
#example data
data[0:5]

In [ ]:
#time step
data['time (us)'].diff()[0:5]

In [ ]:
start=0
stop=len(data)
samplerate=4096.0
NFFT=1024

In [ ]:
show_axes_v2(data,start,stop,samplerate,['debug[0]','debug[1]','debug[2]'],'gyro_scaled',NFFT)
show_axes_v2(data,start,stop,samplerate,['gyroADC[0]','gyroADC[1]','gyroADC[2]'],'filtered gyro',NFFT)

show_axes_v2(data,start,stop,samplerate,['debug[2]'],'gyro_scaled',NFFT)
show_axes_v2(data,start,stop,samplerate,['gyroADC[2]'],'filtered gyro',NFFT)
#show_axes_v2(data,start,stop,samplerate,['motor[0]','motor[1]','motor[2]','motor[3]'],'',NFFT)
#show_axes_v2(data,start,stop,samplerate,['axisD[0]','axisD[1]'],'axisD',NFFT)

In [ ]:
#draw metric against time
plt.figure()
plt.plot(data['time (us)'], data['axisD[0]'])

In [ ]:
calc_max_noise_regression(data,40000,52700,samplerate,'debug[0]','debug[2]',1024)

In [ ]:
series = data['debug[2]'][51000:54200].values
print(series[10:40])
L = 400
ssa = SSA(series, L)
ssa.components_to_df(10).plot()
ssa.orig_TS.plot(alpha=0.4)
plt.xlabel("$t$")
plt.ylabel(r"$\tilde{F}_i(t)$")
plt.title("ssa");

In [ ]:
#correlation
plt.figure()
plt.subplot(1,2,1)
ssa.plot_wcorr()
plt.subplot(1,2,2)
ssa.plot_wcorr(0,12)

In [ ]:
#contribution of components 
ssa.plot_comp_contrib()
#plt.tight_layout()

In [ ]:
groups = [[0,1],[2,3],[4,5],[6],[7,8],[9,10,11,12]]
ssa.plot_groups(groups)
plt.tight_layout()

In [ ]:
plt.figure()
residue_components = [2,3,4,5,7,8]
residue = ssa.reconstruct(residue_components)
residue.plot()
plt.figure()
from scipy import signal
powerSpectrum, freqencies, time, imageAxis = plt.specgram(residue, Fs=samplerate, NFFT=512)
